In [236]:
import numpy as np

# Rabin-Karp

In [339]:
# можно еще искать сразу несколько одинаковой длины, отсортировав предварительно Patterns, доделаю

def RK_search(text, Patterns):
    
    x_RK = 0 # случайный параметр
    
    # Полиномиальный хэш
    def hashh(text, m=0, s='', q=(2**61-1), init=False):
        nonlocal x_RK
        
        if init:
            x_RK = np.random.randint(2, 10**5) # хочу randint(2, q), но не могу совладать с переполнением инта
            print('function "hashh" was initialized by x_RK =', x_RK, '\n')
        elif not x_RK:
            raise BaseException('Initiate hashh function to start work!')

        if not m:
            m = len(text)

        init_arr = list(text[:m])
        pol_coef = np.power(x_RK, np.arange(m)[::-1])
        hs = np.sum(np.vectorize(ord)(init_arr) * pol_coef) % q
        text += '$'
        for i in range(len(text) - m):
            yield hs
            hs = ((hs - ord(text[i])*np.power(x_RK, m-1))*x_RK + ord(text[i+m])) % q
        
    # Поиск
    finds = []
    next(hashh('a', init=True))
    for pat in Patterns:
        f = []
        hs_p = next(hashh(pat))
        i = 0
        for hs in hashh(text, len(pat)):
            if hs == hs_p:
                f.append(i)
            i += 1
        if len(f):
            finds.append(f)
        else:
            finds.append(['not in text'])
            
    return finds

In [340]:
text = 'aaabaababbaa'
Patterns = ['aab', 'ab', 'aaa', 'bab', 'bbb', 'abb']

finds = RK_search(text, Patterns)

print(text, '\n')
for i in range(len(Patterns)):
    print(Patterns[i], '\t:', *finds[i])

function "hashh" was initialized by x_RK = 56022 

aaabaababbaa 

aab 	: 1 4
ab 	: 2 5 7
aaa 	: 0
bab 	: 6
bbb 	: not in text
abb 	: 7


# Knuth–Morris–Pratt

In [253]:
def KMP_search(text, pattern):
    
    # Префикс-функция
    def pref(s):
        n = len(s)
        pr = np.zeros(n, dtype=np.int)
        for i in range(n-1):
            k = i
            while True:
                if s[i+1] == s[pr[k]]:
                    pr[i+1] = pr[k] + 1
                    break
                elif pr[k] == 0:
                    pr[i+1] = 0
                    break
                else:
                    k = pr[k]
        return pr
    
    # Поиск
    m = len(pattern)
    pref_patt = pref(pattern)
    i = 0
    k = 0
    while i in range(len(text)) and k in range(m):
        if text[i] == pattern[k]:
            i += 1
            k += 1
        elif k:
            k = pref_patt[k]
        else:
            i += 1
    if k == m:
        return i - m
    return -1

In [254]:
text = 'shsjkywabcdabscabcdabhskchabcdabscafhjsganabcdabscabcdabiakwjd'
pattern = 'abcdabscabcdabia'

i = KMP_search(text, pattern)
print('i =', i, '\nsubstring =', text[i:i+len(pattern)])

i = 42 
substring = abcdabscabcdabia


# Aho–Corasick

In [232]:
def AC_search(text, Patterns):
    finds = {} # словарь вхождений

    # Создаем бор
    Bor = {'root':{'par':(-1, -1), 'term':-1, 'suf_link':-1, 'up_link':-1}}
    i = 0
    node = 'root'
    pat_ind = 0
    for pat in Patterns:
        finds[pat] = []
        node = 'root'
        for c in pat:
            if c not in Bor[node]:
                Bor[i] = {'par':(node, c), 'term':-1} # par-(родитель, путь из родителя), term-терминальная?
                Bor[node][c] = i
            node = Bor[node][c]
            i += 1
        Bor[node]['term'] = pat_ind # отмечаем конец паттерна как терминальную вершину
        pat_ind += 1


    # Строим суффиксные ссылки и сжатые суффиксные ссылки
    queue = []
    for first_lev_node in Bor['root']: # кладем на стек вершины первого уровня
        if Bor['root'][first_lev_node] in Bor:
            queue.append(Bor['root'][first_lev_node])

    while len(queue):
        node = Bor[queue.pop(0)] # снимаем вершину со стека

        # строим суффиксную ссылку
        par_node = node['par'][0] # смотрим на suf_link родителя
        while True:
            if Bor[par_node]['suf_link'] == -1: # это корень?
                node['suf_link'] = 'root'
                break
            elif node['par'][1] in Bor[Bor[par_node]['suf_link']]: # из suf_link родителя можно пройти в node?
                node['suf_link'] = Bor[Bor[par_node]['suf_link']][node['par'][1]]
                break
            elif Bor[par_node]['par'][0] == -1:
                break
            else:
                par_node = Bor[par_node]['par'][0] # если нет, смотрим на suf_link родителя родителя

        # строим суффиксную ссылку
        suf_link_node = node['suf_link']
        if suf_link_node == 'root':
            node['up_link'] = -1
        elif Bor[suf_link_node]['term'] + 1:
            node['up_link'] = suf_link_node
        else:
            node['up_link'] = Bor[suf_link_node]['up_link']

        # сладываем на стек всех потомков вершины node
        for next_lev_node in node:
            if node[next_lev_node] in Bor and next_lev_node not in ['term', 'suf_link'] :
                queue.append(node[next_lev_node])


    # Выполняем поиск            
    node = 'root'
    for i in range(len(text)):
        while True:
            if text[i] in Bor[node]: # если можем пройти по ребру дальше
                node = Bor[node][text[i]]
                if Bor[node]['term'] + 1: # если это терминальная вершина
                    pat = Patterns[Bor[node]['term']]
                    finds[pat].append(i-len(pat)+1)
                if Bor[node]['up_link'] + 1: # если из этой вершины есть сжатая суфф ссылка
                    pat = Patterns[Bor[Bor[node]['up_link']]['term']]
                    finds[pat].append(i-len(pat)+1)
                break
            elif Bor[node]['suf_link'] + 1: # если суф ссылка не в корень
                node = Bor[node]['suf_link']
            else: # если суф ссылка в корень
                break
                
    return finds

In [252]:
text = 'aaabaababbaa'
Patterns = ['aab', 'ab', 'aaa', 'bab', 'bbb', 'abb', 'ac']

finds = AC_search(text, Patterns)

print(text, '\n')
for i in finds.items():
    print(i[0], '\t:', end=' ')
    print(*i[1]) if  len(i[1]) else print('not in text')

aaabaababbaa 

aab 	: 1 4
ab 	: 2 5 7
aaa 	: 0
bab 	: 6
bbb 	: not in text
abb 	: 7
ac 	: not in text
